# Download a Batch

Use this workflow notebook to download a batch from the Infinity API. 

This is your notebook to edit as needed. 

[Link to Infinity documentation](https://docs.infinity.ai/index.html)

In [ ]:
import os
import re
import numpy as np
import pandas as pd
import infinity_tools.visionfit.api as api

from pathlib import Path
from infinity_core.data_structures import JobType
from infinity_tools.visionfit.vis import summarize_batch_results_as_dataframe, visualize_batch_results
from infinity_tools.common.vis.images import get_subdirectories, view_previews
from visionfit.interfaces import BatchDownloadInterface
from visionfit.utils.notebook import display_video_as_gif
from visionfit.interfaces.common import is_batch_exist
pd.options.display.max_columns = None

In [ ]:
# User Overrides: Customize key parameters. Change the BATCH_ID to download a different batch.
# TOKEN = "YOUR_TOKEN_HERE"
# GENERATOR = "GENERATOR_HERE"
# SERVER = "SERVER_HERE"
# BATCH_ID = "BATCH_ID_HERE"

In [ ]:
# Ensure that a valid BATCH_ID was provided.
if not BATCH_ID or not is_batch_exist(BATCH_ID, TOKEN, GENERATOR, SERVER):
    print("\033[91mEnter valid BATCH_ID you want to download.")

### Get the batch metadata

Create a Session to interact with the API. Then, get the batch metadata with the BATCH_ID. 

In [ ]:
is_downloaded = False
sesh = api.VisionFitSession(token=TOKEN, generator=GENERATOR, server=SERVER)
batch = sesh.batch_from_api(batch_id=BATCH_ID)
df = pd.DataFrame.from_records(batch.job_params)
df.head()

### Download the batch

Download the data from your batch into a folder on your local machine. The Download button will download all completed jobs. Press the button multiple times to download any new completed jobs that are not already present on your local machine.

In [ ]:
downloads_path = Path("downloads")
downloads_path.mkdir(parents=True, exist_ok=True)
batch_name = re.sub(r"[^\w_. -]", "_", batch.name)
batch_path = downloads_path / f"{batch_name}/batch_ID_{batch.batch_id}"
batch_path.mkdir(parents=True, exist_ok=True)
download_interface = BatchDownloadInterface(batch, batch_path)

### Inspect and visualize

After the batch has been successfully downloaded, inspect both the data and its metadata. 

In [ ]:
is_downloaded = batch_path.exists()
if is_downloaded:
    df = summarize_batch_results_as_dataframe(batch_path) 
    display(df)

In [ ]:
if is_downloaded:
    if batch.job_type == JobType.PREVIEW:
        preview_job_paths = get_subdirectories(batch_path)
        view_previews(preview_job_paths if len(preview_job_paths) < 20 else np.random.choice(preview_job_paths, 20, replace=False))
    else:
        display_video_as_gif(df.sample().iloc[0].job_path + '/video.mp4')

In [ ]:
if is_downloaded:
    visualize_batch_results(batch_path)